In [1]:
import yaml
import geometry as geo
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt
from math import sqrt

import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html

def draw_graph(G0, pos=None):
    pos = pos or nx.spring_layout(G0)
    plt.figure(figsize=(12, 12))
    nx.draw(G0,pos,labels={node:node for node in G0.nodes()})
    def edge_label(a, b):
        datas = G0.get_edge_data(a, b)
        s = '%d edge%s' % (len(datas), 's' if len(datas)>=2 else '')
        for k, v in datas.items():
            if v:
                if 'label' in v:
                    s += '\n %s' % v['label']
                else:
                    s += '\n %s' %v
        return s
    edge_labels = dict([ ((a,b), edge_label(a,b)) for a,b in G0.edges()])
    nx.draw_networkx_edge_labels(G0,pos,edge_labels=edge_labels,font_color='red')
    plt.axis('off')
    plt.show()

INFO:geometry:version: 2.0.5
INFO:commons:version: 6.1.5
INFO:typing:version: 6.1.8
DEBUG:duckietown_world:duckietown-world version 6.2.7 path /home/chrigi/Master-Thesis-Code/driving-games/venv/lib/python3.8/site-packages


In [2]:
map_data_yaml = """
tile_size: 1

tiles:
  - [curve_left/W , straight/W, 3way_left/W   , straight/W, 3way_left/W   , straight/W  , curve_left/N]
  - [straight/S   , asphalt   , straight/N    , asphalt   , straight/N    , asphalt     ,  straight/N]
  - [3way_left/S  , straight/W, 4way          , straight/E, 4way          , straight/E  ,  3way_left/N]
  - [straight/S   , asphalt   , straight/S    , asphalt   , straight/S    , asphalt     ,  straight/N]
  - [3way_left/S  , straight/W, 4way          , straight/E, 4way          , straight/E  ,  3way_left/N]
  - [straight/S   , asphalt   , straight/S    , asphalt   , straight/S    , asphalt     ,  straight/N]
  - [curve_left/S , straight/E, 3way_left/E   , straight/E, 3way_left/E   , straight/E  , curve_left/E]

"""

map_data = yaml.load(map_data_yaml, Loader=yaml.SafeLoader)

m = dw.construct_map(map_data)

ipython_draw_html(m);

INFO:duckietown_world: data: /home/chrigi/Master-Thesis-Code/driving-games/venv/lib/python3.8/site-packages/duckietown_world/data
INFO:duckietown_world: area: RectangularArea(pmin=[0. 0.],pmax=[6.908 6.908])
DEBUG:commons.fs:Written 0.7MB to: out/ipython_draw_html/140201960664224/drawing.html
INFO:duckietown_world:Written SVG
 fn_svg: out/ipython_draw_html/140201960664224/drawing.svg


In [3]:
map_data_yaml = """
tile_size: 1

tiles:
  - [asphalt]

"""

map_data = yaml.load(map_data_yaml, Loader=yaml.SafeLoader)

m = dw.construct_map(map_data)

tile_size=map_data["tile_size"]

width_old = 0.376
width = width_old / 2

width_between_lanes_old = 0.22 * 2 - width_old
width_between_lanes = width_between_lanes_old / 2
# y1 = -0.365
# y2 = -0.115
y2 = -(width_between_lanes / 2 + width / 2)
y1 = y2 - (width_between_lanes + width)

y3, y4 = -y2, -y1

x1, x2, x3, x4 = y1, y2, y3, y4

half_tile = tile_size / 2

# double straight
ctr_pts1 = [
    dw.SE2Transform([-half_tile, y1], np.deg2rad(0)),
    dw.SE2Transform([half_tile, y1], np.deg2rad(0)),
]
ctr_pts2 = [
    dw.SE2Transform([-half_tile, y2], np.deg2rad(0)),
    dw.SE2Transform([half_tile, y2], np.deg2rad(0)),

]

def midpoints(start, end):
    x1, y1 = start
    x2, y2 = end
    
    r = x2 - x1
    dr = (1-sqrt(2)/2) * r
    if y2 > y1:
        res = [x2 - dr, y1 + dr]
    else:
        res = [x2 - dr, y1 - dr]

    return [round(_, 3) for _ in res]
    
    
#double left
start = [-half_tile, y1]
end = [x4, half_tile]
middle = midpoints(start, end)
ctr_pts3 = [
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(45)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

start = [-half_tile, y2]
end = [x3, half_tile]
middle = midpoints(start, end)
ctr_pts4 = [
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(45)),
    dw.SE2Transform(end, np.deg2rad(90)),
]

# double right
start = [-half_tile, y1]
end = [x1, -half_tile]
middle = midpoints(start, end)
ctr_pts5 = [
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(-45)),
    dw.SE2Transform(end, np.deg2rad(-90)),
]

start = [-half_tile, y2]
end = [x2, -half_tile]
middle = midpoints(start, end)
ctr_pts6 = [
    dw.SE2Transform(start, np.deg2rad(0)),
    dw.SE2Transform(middle, np.deg2rad(-45)),
    dw.SE2Transform(end, np.deg2rad(-90)),
]


test_lane1 = dw.LaneSegment(width, ctr_pts1)
test_lane2 = dw.LaneSegment(width, ctr_pts2)
test_lane3 = dw.LaneSegment(width, ctr_pts3)
test_lane4 = dw.LaneSegment(width, ctr_pts4)
test_lane5 = dw.LaneSegment(width, ctr_pts5)
test_lane6 = dw.LaneSegment(width, ctr_pts6)

gt = dw.SE2Transform([half_tile, half_tile], 0)
m.set_object("Test Lane 1", test_lane1, ground_truth=gt)
m.set_object("Test Lane 2", test_lane2, ground_truth=gt)
m.set_object("Test Lane 3", test_lane3, ground_truth=gt)
m.set_object("Test Lane 4", test_lane4, ground_truth=gt)
m.set_object("Test Lane 5", test_lane5, ground_truth=gt)
m.set_object("Test Lane 6", test_lane6, ground_truth=gt)
ipython_draw_html(m);

INFO:duckietown_world: area: RectangularArea(pmin=[0. 0.],pmax=[1. 1.])
DEBUG:commons.fs:Written 0.1MB to: out/ipython_draw_html/140200296160080/drawing.html
INFO:duckietown_world:Written SVG
 fn_svg: out/ipython_draw_html/140200296160080/drawing.svg
